In [1]:
import cvxpy as cp
import numpy as np

m = 10
n = 5

np.random.seed(1)
A = np.random.randn(m,n)
b = np.random.randn(m)

x = cp.Variable(n)
cost = cp.sum_squares(A*x - b)
objective = cp.Minimize(cost)
constraints = [0 <= x, x <=1]
prob= cp.Problem(objective, constraints)

result = prob.solve()

print("optimal parameter:\n", x.value)
print("Lagrange parameter\n", constraints[0].dual_value)
print("status:"+ prob.status)

optimal parameter:
 [-4.95922264e-21  6.07571976e-21  1.34643668e-01  1.24976681e-01
 -4.57130806e-21]
Lagrange parameter
 [2.00105768 0.75536127 0.         0.         1.17911779]
status:optimal


In [2]:
import cvxopt
from cvxopt import matrix
import numpy as np

P = matrix(np.diag([1.0, -1.0]))
q = matrix(np.array([3.0, 4.0]))
A = matrix(np.array([[-1.0, 0.0],[2.0,3.0]]).astype(np.float))
b = matrix(np.array([1.0,1.0]))
G = matrix(np.array([[-2.0, 0.0]]).astype(np.float))
h = matrix(np.array([3.0]))

sol = cvxopt.solvers.qp(P,q, A=A, b=b,G=G, h=h)
print(sol)
print(sol['x'])
print(sol['primal objective'])

     pcost       dcost       gap    pres   dres
 0:  1.0000e+00  8.8818e-16  1e+00  4e-16  8e-01
 1:  1.0000e+00  9.9000e-01  1e-02  8e-17  8e-03
 2:  1.0000e+00  9.9990e-01  1e-04  1e-16  8e-05
 3:  1.0000e+00  1.0000e+00  1e-06  1e-16  8e-07
 4:  1.0000e+00  1.0000e+00  1e-08  7e-17  8e-09
Optimal solution found.
{'x': <2x1 matrix, tc='d'>, 'y': <2x1 matrix, tc='d'>, 's': <1x1 matrix, tc='d'>, 'z': <1x1 matrix, tc='d'>, 'status': 'optimal', 'gap': 1.0000000000000037e-08, 'relative gap': 1.0000000100000038e-08, 'primal objective': 1.0, 'dual objective': 0.99999999, 'primal infeasibility': 7.401486830834377e-17, 'dual infeasibility': 8.000000064507973e-09, 'primal slack': 1.0000000000000002, 'dual slack': 1.0000000000000035e-08, 'iterations': 4}
[-1.00e+00]
[ 1.00e+00]

1.0


In [3]:
import cvxpy as cp

# Create 2 scalar optimization value
x = cp.Variable()
y = cp.Variable()

# Create 2 constrains
constrains = [ (x + y) == 1,
               (x - y) >= 1]

# Form objective(目的関数・コスト関数)
cost = (x- y)**2
obj = cp.Minimize(cost)

# Form and solve problem
prob = cp.Problem(obj, constrains)
prob.solve()

# Result
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal value", x.value, y.value)


status: optimal
optimal value 1.0
optimal value 1.0 1.570086213240983e-22


In [4]:
import cvxpy as vp

# Creaate 2 scalar optimization value
x1 = cp.Variable()
x2 = cp.Variable()

# Create 2 constraints
constraints = [(x1 + x2) == 1,
               (x1 - x2) >= 1]

# Form objective
cost = -x1+x2
obj = cp.Minimize(cost)

# Form and solve problem
prob = cp.Problem(obj, constraints)
prob.solve()

# Result
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal value", x.value, y.value)

status: unbounded
optimal value -inf
optimal value 1.0 1.570086213240983e-22


In [5]:
import cvxpy as cp

# Create Variables
x = cp.Variable()
y = cp.Variable()

# Create constraints
constraints = [(x + y) >= 1]

# Cost function
cost = x**2 + y**2
# Form objective
obj = cp.Minimize(cost)

# Form and solve problem
prob = cp.Problem(obj,constraints)
prob.solve()

# Result
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal value", x.value, y.value)


status: optimal
optimal value 0.5000000000000002
optimal value 0.5000000000000001 0.5000000000000001


In [6]:

import cvxpy as cp

# Create Variables
x1 = cp.Variable()
x2 = cp.Variable()

# Create constraints
constraints = [x1 <= 5,
               -x1<= 5,
               x2 <= 5,
               -x2<= 5]

# Cost
cost = (x1 - 6)**2 + 2+x2**2
# objective
obj = cp.Minimize(cost)

# Form and solve problem
prob = cp.Problem(obj, constraints)
prob.solve()

# Result
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal value", x1.value, x2.value)


status: optimal
optimal value 3.000000000000001
optimal value 5.0 0.0


# Model predoctive control with cvxpy

In [7]:
import time
from cvxpy import *
import numpy as np
import matplotlib.pyplot as plt
print("Simulation start")

np.random.seed(1)
n = 4   #state size
m = 2   #input size
T = 50  #number of horizon

#simulation parameter
alpha = 0.2
beta = 5.0

# Model Parameter
A = np.eye(n) + alpha*np.random.randn(n,n)
B = np.random.randn(n,m)
x_0 = beta*np.random.randn(n,1)

x = cp.Variable(shape=(n, T+1))
u = cp.Variable(shape=(m, T))

states = []
for t in range(T):
    cost = sum_squares(x[:,t+1]) + sum_squares(u[:,t])
    constr = [x[:,t+1] == A*x[:,t] + B*u[:,t],
              norm(u[:,t], 'inf') <= 1]
    states.append( Problem(Minimize(cost), constr) )
# sums problem objectives and concatenates constraints.
prob = sum(states)
#prob.constraints = [x[:,0] == x_0]
#prob.constraints += [x[:,T+1] == 0,
#                     x[:,0] == x_0]

start = time.time()
result=prob.solve(verbose=True)
elapsed_time = time.time() - start
print ("calc time:{} [s]".format(elapsed_time))

if result == float("inf"):
    print("Cannot optimize")
    import sys
    sys.exit()

f = plt.figure()
ax = f.add_subplot(211)
u1=np.array(u[0,:].value[0,:])[0].tolist()
u2=np.array(u[1,:].value[0,:])[0].tolist()
plt.plot(u1,'-r',label="u1")
plt.plot(u2,'-b',label="u2")
plt.ylabel(r"$u_t$", fontsize=16)
plt.yticks(np.linspace(-1.0, 1.0, 3))
plt.legend()
plt.grid(True)

plt.subplot(2,1,2)
x1=np.array(x[0,:].value[0,:])[0].tolist()
x2=np.array(x[1,:].value[0,:])[0].tolist()
x3=np.array(x[2,:].value[0,:])[0].tolist()
x4=np.array(x[3,:].value[0,:])[0].tolist()
plt.plot(range(T+1), x1,'-r',label="x1")
plt.plot(range(T+1), x2,'-b',label="x2")
plt.plot(range(T+1), x3,'-g',label="x3")
plt.plot(range(T+1), x4,'-k',label="x4")
plt.yticks([-25, 0, 25])
plt.ylim([-25, 25])
plt.ylabel(r"$x_t$", fontsize=16)
plt.xlabel(r"$t$", fontsize=16)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

Simulation start
-----------------------------------------------------------------
           OSQP v0.5.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2018
-----------------------------------------------------------------
problem:  variables n = 654, constraints m = 750
          nnz(P) + nnz(A) = 2750
settings: linear system solver = qdldl,
          eps_abs = 1.0e-04, eps_rel = 1.0e-04,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on

objective    pri res    dua res    rho        time
   1   0.0000e+00   0.00e+00   0.00e+00   1.00e-01   5.90e-04s
  25   0.0000e+00   0.00e+00   0.00e+00   1.00e-01   1.31e-03s

status:               solved
solution polish:     

IndexError: too many indices for array